In [2]:
import pandas as pd

df = pd.read_csv("funnel_data.csv")

df["timestamp"] = pd.to_datetime(df["timestamp"])
df_sorted = df.sort_values(["user_id","timestamp"])
df.columns

Index(['user_id', 'timestamp', 'event_name', 'session_id', 'device',
       'page_url', 'duration'],
      dtype='object')

# Journey and Segmentation

In [19]:

journey_df = df_sorted.groupby('user_id')["event_name"].apply(lambda x: " → ".join(x)).reset_index()
journey_df.columns = ["user_id","event_path"]

def segment_user(path):
    if 'purchase_complete' in path:
        return 'Purchaser'
    elif 'cart_add' in path:
        return 'Cart Abandoner'
    elif 'product_click' in path:
        return 'Interested Visitor'
    elif 'homepage_view' in path:
        return 'Bouncer'
    else:
        return 'Undefined'
    
journey_df['segment'] = journey_df["event_path"].apply(segment_user)
journey_df.head()

,user_id,event_path,segment
0,user_1,homepage_view → homepage_view → cart_add → hom...,Cart Abandoner
1,user_10,cart_add → cart_add → homepage_view → product_...,Cart Abandoner
2,user_100,cart_add → homepage_view → homepage_view → pur...,Purchaser
3,user_1000,homepage_view → checkout_start → homepage_view...,Purchaser
4,user_101,product_click → homepage_view → homepage_view ...,Interested Visitor


# PERSONA

In [27]:
cart_users = journey_df[journey_df['segment'] == 'Cart Abandoner']

cart_users_ids = set(cart_users['user_id'])

cart_data = df[df['user_id'].isin(cart_users_ids)].copy()

In [29]:
top_device = cart_data['device'].mode()[0]

cart_data['hour'] = cart_data['timestamp'].dt.hour
top_hour = cart_data['hour'].mode()[0]

In [34]:
top_event = cart_data['event_name'].mode()[0]
avg_duration = round(cart_data['duration'].mean())
top_location = cart_data['location'].mode()[0] if 'location' in cart_data.columns else 'Unknown'

In [40]:
persona = {
    "Adı": "Sepetçi Seda",
    "Segment": "Cart Abandoner",
    "Cihaz": top_device,
    "En Aktif Saat": f"{top_hour}:00 civarı",
    "Ortalama sure": f"{avg_duration} dakika",
    "Davranış": f"En sık yaptığı event: {top_event}",
    "Lokasyon": top_location,
    "Strateji": "Sepet hatırlatma maili + indirim önerisi"
}
# print(persona)

for k, v in persona.items():
    print(f"{k:<15}: {v}")



Adı            : Sepetçi Seda
Segment        : Cart Abandoner
Cihaz          : desktop
En Aktif Saat  : 8:00 civarı
Ortalama sure  : 93 dakika
Davranış       : En sık yaptığı event: homepage_view
Lokasyon       : Unknown
Strateji       : Sepet hatırlatma maili + indirim önerisi
